# Carregamento de Dados

Objetivos dessa aula:
* Carregar um dataset customizado
* Implementar o fluxo de treinamento **e validação** completo de uma rede


## Hiperparâmetros

Vamos manter a organização do último script :)

* imports de pacotes
* configuração de hiperparâmetros
* definição do hardware padrão utilizado

E bora de GPU de novo! 


In [1]:
import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import time
import requests
import zipfile


import matplotlib.pyplot as plt

# Configurando hiperparâmetros.
args = {
    'epoch_num': 200,     # Número de épocas.
    'lr': 5e-5,           # Taxa de aprendizado.
    'weight_decay': 5e-4, # Penalidade L2 (Regularização).
    'num_workers': 0,     # Número de threads do dataloader.
    'batch_size': 20,     # Tamanho do batch.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


## Dataset 

Dataset de aplicativos para aluguel de bicicletas (*Bike Sharing Dataset*). <br>
* Dadas algumas informações como velocidade do vento, estação do ano, etc., quantas bicicletas serão alugadas na próxima hora?

Esse é um problema de **Regressão**, onde precisamos estimar uma variável dependente em um espaço contínuo (alugueis de bikes) a partir de um conjunto de variáveis independentes (as condições no momento).

### Baixando o dataset

Fonte: https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset



In [2]:
url =  'https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip'
r = requests.get(url)

with open('dados/bicicletas/bike-sharing.zip', 'wb') as f:
    f.write(r.content)

In [3]:
with zipfile.ZipFile('dados/bicicletas/bike-sharing.zip', 'r') as zip_ref:
    zip_ref.extractall('dados/bicicletas/')

### Visualizando os dados

In [2]:
data_frame = pd.read_csv('dados/bicicletas/hour.csv')
data_frame

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


### Tratamento de dados

**Variáveis Categóricas** <br>
Como descrito na página do dataset, apenas as variáveis numéricas estão normalizadas. No caso das categóricas (como dia da semana e estação do ano), cada elemento contém o índice da categoria.

Existem várias formas de lidar com variáveis categóricas em uma regressão, mas para não desviar o foco da nossa aula manteremos os valores originais das variáveis categóricas.

**Separação em treino e teste**<br>

Para treinar e validar o nosso modelo, precisamos de dois conjuntos de dados (treino e teste). Para isso, utilizaremos a função ```torch.randperm``` para amostrar aleatoriamente um percentual dos dados, separando-os para validação.

Documentação: https://pytorch.org/docs/stable/torch.html#torch.randperm

In [3]:
df_train, df_test = train_test_split(data_frame, test_size=0.2)
print(f'Treino: {df_train.shape[0]}, Teste: {df_test.shape[0]}')

df_train.to_csv('dados/bicicletas/df_train.csv', index=False)
df_test.to_csv('dados/bicicletas/df_test.csv', index=False)

Treino: 13903, Teste: 3476


### Classe Dataset

O pacote ```torch.util.data``` possui a classe abstrata ```Dataset```. Ela permite que você implemente o seu próprio dataset reescrevendo os métodos:

* ```__init__(self)```: Define a lista de amostras do seu dataset
* ```__getitem__(self, idx)```: Carrega uma amostra, aplica as devidas transformações e retorna uma **tupla ```(dado, rótulo)```**.
* ```__len__(self)```: Retorna a quantidade de amostras do dataset

Tutorial completo do PyTorch: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html


In [10]:
class Bicicletas(Dataset):
  def __init__(self, csv_path):

    self.dados = pd.read_csv(csv_path).to_numpy()
  
  def __getitem__(self, idx):

    if torch.is_tensor(idx):
      idx = idx.tolist()

    sample = self.dados[idx][2:14]
    label = self.dados[idx][-1:]
    
    # Converte para tensor
    features = torch.from_numpy(sample.astype(np.float32))
    label = torch.from_numpy(label.astype(np.float32))

    sample = (features, label)

    return sample

  def __len__(self):
    return len(self.dados)

In [11]:
train_set = Bicicletas('dados/bicicletas/df_train.csv')
test_set = Bicicletas('dados/bicicletas/df_test.csv')

dado, rotulo = train_set[0]
print(rotulo)
print(dado)

tensor([86.])
tensor([ 4.0000,  0.0000, 10.0000,  6.0000,  0.0000,  1.0000,  1.0000,  1.0000,
         0.2400,  0.2879,  0.8700,  0.0000])


### Construindo conjuntos de treino e teste

In [12]:
print('Tamanho do treino: ' + str(len(train_set)) + ' amostras')
print('Tamanho do teste: ' + str(len(test_set)) + ' amostras')

Tamanho do treino: 13903 amostras
Tamanho do teste: 3476 amostras


## Dataloader


In [13]:
# Criando dataloader
train_loader = DataLoader(train_set,
                          batch_size = args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         batch_size = args['batch_size'],
                         num_workers=args['num_workers'],
                         shuffle=False)

In [15]:
for batch in train_loader:
    dado, label = batch
    print(dado.size())
    print(label.size())
    break

torch.Size([20, 12])
torch.Size([20, 1])


O objeto retornado é um **iterador**, podendo ser utilizado para iterar em loops mas não suportando indexação.

## Implementando o MLP

Essa parte aqui você já tira de letra! Minha sugestão é construir um modelo com:

* **Duas camadas escondidas**. Lembre-se de alternar as camadas com ativações não-lineares. 
* Uma camada de saída (com qual ativação?)

In [16]:
class MLP(nn.Module):
  
  def __init__(self, input_size, hidden_size, out_size):
    super(MLP, self).__init__()
    
    self.features = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
    )
    
    self.classifier = nn.Sequential(
        nn.Linear(hidden_size, out_size),
        nn.ReLU(),
    )

  def forward(self, X):
    
    hidden = self.features(X)
    output = self.classifier(hidden)
    
    return output

input_size  = train_set[0][0].size(0)
hidden_size = 128
out_size    = 1

net = MLP(input_size, hidden_size, out_size).to(args['device'])
print(net)

MLP(
  (features): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=1, bias=True)
    (1): ReLU()
  )
)


## Definindo loss e otimizador

Se lembra quais as funções de perda adequadas para um problema de regressão?

In [21]:
criterion = nn.L1Loss().to(args['device'])

optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

# Fluxo de Treinamento & Validação

## Treinamento

Relembrando o passo a passo do fluxo de treinamento:
* Iterar nas épocas
* Iterar nos batches
* Cast dos dados no dispositivo de hardware
* Forward na rede e cálculo da loss
* Cálculo do gradiente e atualização dos pesos

Esse conjunto de passos é responsável pelo processo iterativo de otimização de uma rede. **A validação** por outro lado, é apenas a aplicação da rede em dados nunca antes visto para estimar a qualidade do modelo no mundo real.

## Validação

Para essa etapa, o PyTorch oferece dois artifícios:
* ```model.eval()```: Impacta no *forward* da rede, informando as camadas caso seu comportamento mude entre fluxos (ex: dropout).
* ```with torch.no_grad()```: Gerenciador de contexto que desabilita o cálculo e armazenamento de gradientes (economia de tempo e memória). Todo o código de validação deve ser executado dentro desse contexto.

Exemplo de código para validação

```python
net.eval()
with torch.no_grad():
  for batch in test_loader:
      # Código de validação
```


Existe o equivalente ao ```model.eval()``` para explicitar que a sua rede deve estar em modo de treino, é o ```model.train()```. Apesar de ser o padrão dos modelos, é boa prática definir também o modo de treinamento.

In [25]:
def train(train_loader, net, epoch):
    net.train()
    epoch_loss = list()
    for batch in train_loader:

        dado, rotulo = batch

        # Cast GPU
        dado = dado.to(args['device'])
        rotulo = rotulo.to(args['device'])

        # Forward
        predicao = net(dado)
        loss = criterion(predicao, rotulo)
        epoch_loss.append(loss.cpu().data)

        # Backward 
        loss.backward()
        optimizer.step()

    epoch_loss = np.asarray(epoch_loss)
    print(f'Época {epoch}, Loss: {epoch_loss.mean()} +/- {epoch_loss.std()}, Tempo: ')


In [26]:
def validate(test_loader, net, epoch):

  # Evaluation mode
  net.eval()
  
  start = time.time()
  
  epoch_loss  = []
  
  with torch.no_grad(): 
    for batch in test_loader:

      dado, rotulo = batch

      # Cast do dado na GPU
      dado = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward
      ypred = net(dado)
      loss = criterion(ypred, rotulo)
      epoch_loss.append(loss.cpu().data)

  epoch_loss = np.asarray(epoch_loss)
  
  end = time.time()
  print('********** Validate **********')
  print('Epoch %d, Loss: %.4f +/- %.4f, Time: %.2f\n' % (epoch, epoch_loss.mean(), epoch_loss.std(), end-start))
  
  return epoch_loss.mean()
    

In [28]:
for epoch in range(args['epoch_num']):
  train(train_loader, net, epoch)
  validate(test_loader, net, epoch)

Época 64, Loss: 77.1151351928711 +/- 22.043760299682617, Tempo: 
********** Validate **********
Epoch 64, Loss: 77.4810 +/- 22.5750, Time: 0.14

Época 65, Loss: 75.9288330078125 +/- 19.61199951171875, Tempo: 
********** Validate **********
Epoch 65, Loss: 73.9550 +/- 19.2312, Time: 0.14

Época 66, Loss: 75.61204528808594 +/- 22.748016357421875, Tempo: 
********** Validate **********
Epoch 66, Loss: 73.6869 +/- 20.1000, Time: 0.15

Época 67, Loss: 74.74018859863281 +/- 18.52726936340332, Tempo: 
********** Validate **********
Epoch 67, Loss: 73.7058 +/- 20.5276, Time: 0.18

Época 68, Loss: 74.63904571533203 +/- 23.575101852416992, Tempo: 
********** Validate **********
Epoch 68, Loss: 74.0031 +/- 18.8825, Time: 0.20

Época 69, Loss: 74.53748321533203 +/- 20.380413055419922, Tempo: 
********** Validate **********
Epoch 69, Loss: 76.4153 +/- 21.8237, Time: 0.22

Época 70, Loss: 74.44606018066406 +/- 20.827392578125, Tempo: 
********** Validate **********
Epoch 70, Loss: 74.8941 +/- 18.585

# Gráfico de convergência